In [380]:
import base64
import numpy as np
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

def valid_pkcs7(plain):
    return 1 <= plain[-1] <= 16 and plain[-plain[-1]:] == bytes([plain[-1]]) * plain[-1]

def unpkcs7(plain):
    if 1 <= plain[-1] <= 16 and plain[-plain[-1]:] == bytes([plain[-1]]) * plain[-1]:
        return plain[:-plain[-1]]
    else:
        raise ValueError("Bad padding")

B = lambda s: [s[i:i+16] for i in range(0, len(s), 16)]

def change_byte(b, pos, B):
    pos = pos % len(B)
    return B[:pos] + b + B[pos+1:]

def fixed_xor(a, b):
    return bytes([_a ^ _b for _a, _b in zip(a, b)])

In [14]:
K = np.random.bytes(16)

In [386]:
def server_cipher(key=K):

    P = b"""MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=
    MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=
    MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==
    MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==
    MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl
    MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==
    MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==
    MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=
    MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=
    MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93""".splitlines()
    P = [base64.decodebytes(p) for p in P]

    padder = padding.PKCS7(128).padder()
    prefix = np.random.choice(P)
    plain = padder.update(prefix) + padder.finalize()

    iv = np.random.bytes(16)
    aes = Cipher(algorithms.AES128(key), modes.CBC(iv)).encryptor()
    ciphertxt = aes.update(plain) + aes.finalize()

    return iv, ciphertxt

In [124]:
def padding_oracle(iv, ciphertxt, key=K):
    aes = Cipher(algorithms.AES128(key), modes.CBC(iv)).decryptor()
    plain = aes.update(ciphertxt) + aes.finalize()
    return valid_pkcs7(plain)

In [224]:
iv, ciphertxt = server_cipher()

In [349]:
def break_P2(C0, C1, C2, padding_oracle):

    P2 = b''
    C1_blkp = C1

    for i in range(256):
        C1_prime = change_byte(bytes([i]), -1, C1)
        if padding_oracle(C0, C1_prime + C2):
            C1_second = change_byte(bytes([(C1_prime[-2] >> 1) ^ C1_prime[-2]]), -2, C1_prime)
            if padding_oracle(C0, C1_second + C2):
                pad_byte = bytes([C1_prime[-1] ^ 0x01 ^ C1[-1]])

    P2 = pad_byte + P2

    for i in range(1, 16):

        C1 = C1_blkp

        for j in range(1, i+1):
            C1 = change_byte(bytes([P2[-j] ^ C1[-j] ^ (i+1)]), -j, C1)

        for k in range(256):
            C1_prime = change_byte(bytes([k]), -(i+1), C1)
            if padding_oracle(C0, C1_prime + C2):
                pad_byte = bytes([C1_prime[-(i+1)] ^ (i+1) ^ C1[-(i+1)]])

        P2 = pad_byte + P2
    
    return P2


In [364]:
C = B(ciphertxt)
break_P2(np.random.bytes(16), iv, C[0], padding_oracle)

b'000002Quick to t'

In [383]:
def padding_oracle_attack(iv, ciphertxt, padding_oracle):

    blks = [np.random.bytes(16), iv] + B(ciphertxt)

    return unpkcs7(b''.join([break_P2(blks[i], blks[i+1], blks[i+2], padding_oracle) for i in range(len(blks) - 3 + 1)]))

In [384]:
padding_oracle_attack(iv, ciphertxt, padding_oracle)

b'000002Quick to the point, to the point, no faking'

In [393]:
padding_oracle_attack(*server_cipher(), padding_oracle)

b'000009ith my rag-top down so my hair can blow'